In [12]:
from pathlib import Path
from rich.console import Console
import polars as pl
import json

cons = Console()

In [13]:
inppatient_path = Path("../data/mdace/Inpatient")
profee_path = Path("../data/mdace/Profee")

assert inppatient_path.exists(), "The specified path does not exist."
assert profee_path.exists(), "The specified path does not exist."


## Load the JSON Dataset

In [9]:
json_files = list((inppatient_path / "json").glob("*.json")) + list(profee_path.glob("*.json"))
assert len(json_files) > 0, "No JSON files found in the specified paths."
print(f"Found {len(json_files)} JSON files in the specified paths.")

Found 302 JSON files in the specified paths.


In [10]:
sample_datum = json.loads(json_files[0].read_text())
print(json_files[0])

../data/mdace/Inpatient/json/130520-ICD-10.json


In [11]:
cons.print(sample_datum['notes'][0])

{
    'note_id': 1250764,
    'category': 'Radiology',
    'description': 'TUBE CHOLANGIOGRAM',
    'annotations': [
        {
            'begin': 391,
            'end': 404,
            'code': 'BF10YZZ',
            'code_system': 'ICD-10-PCS',
            'description': 'Fluoroscopy of Bile Ducts using Other Contrast',
            'type': 'Human',
            'covered_text': 'CHOLANGIOGRAM'
        }
    ],
    'text': "[**2194-8-20**] 11:48 AM\n BILIARY CATH CHECK                                              Clip # 
[**Clip Number (Radiology) 2905**]\n Reason: Please evaluate anatomy via drain study of PTBD.\n Admitting 
Diagnosis: CHRONIC DISTAL COMMON BILE DUCT/SDA\n  Contrast: OPTIRAY Amt: 20\n ********************************* CPT
Codes ********************************\n * [**Numeric Identifier 162**] TUBE CHOLANGIOGRAM              -58 SERVIC 
BY SAME MD DURING POST OP   *\n ****************************************************************************\n 
______________________________________________________________________________\n [**Hospital 2**] MEDICAL 
CONDITION:\n  43 year old woman s/p open chole and choledocoduodenostomy, with prior PTBD in\n  place which has 
been capped for past 2 days. Please evaluate anatomy with drain\n  study.\n REASON FOR THIS EXAMINATION:\n  Please 
evaluate anatomy via drain study of PTBD.\n 
______________________________________________________________________________\n                                 
FINAL REPORT\n INDICATION:  43-year-old female with prior episode of pancreatitis obstructing\n the common bile 
duct, now status post surgical choledochoduodenostomy, with\n PTBD from right lobe left in place.  Please evaluate 
anatomy and to\n discontinue tube if possible.\n\n PHYSICIAN:  [**Last Name (NamePattern4) **], M.D., attending, 
was present and supervising.  [**First Name8 (NamePattern2) 60**]\n [**Last Name (NamePattern1) 61**], M.D., 
fellow, was performing the procedure.\n\n FLUOROSCOPY TIME:  1.4 minutes.\n\n MEDICATIONS:  Moderate sedation was 
provided by administering divided doses of\n Versed totaling 2 mg throughout the total intraservice time of 40 
minutes,\n during which the patient's hemodynamic parameters were continuously monitored.\n Patient remained in 
control with her PCA device.\n\n PROCEDURES:  Gravity cholangiogram via indwelling right lobe biliary drain.\n 
Removal of drain.\n\n PROCEDURE DETAILS:  Informed consent was obtained from the patient.  She was\n positioned 
supine on the angiography table.  The area was prepped and draped\n in sterile fashion.  Appropriate timeout was 
performed.  Fluoroscopy was used\n intermittently.\n\n With gravity, contrast was dripped into the indwelling 
drain.  This showed\n adequate flow through the choledochoduodenostomy into the duodenum.  There was\n no 
appreciable flow through the ampulla.  The drain was then pulled back\n slightly and more contrast was 
administered.  No intrahepatic strictures were\n evident.  There was free flow of contrast into the bowel.  The 
drain was then\n removed.  Pressure was held for about 15 minutes until there was no longer\n leaking along the 
tract.  A pressure dressing was applied.  Patient left the\n department in stable condition without any immediate 
complication.\n\n FINDINGS:  Patent choledochoduodenostomy tract with free flow from the upper\n common bile duct 
into the duodenum.  The ampulla appears to be fully\n obstructed.  There was no appreciable flow along this 
anatomic pathway, though\n this is likely just higher resistance than the bypass.  No intrahepatic\n               
(Over)\n\n [**2194-8-20**] 11:48 AM\n BILIARY CATH CHECK                                              Clip # 
[**Clip Number (Radiology) 2905**]\n Reason: Please evaluate anatomy via drain study of PTBD.\n Admitting 
Diagnosis: CHRONIC DISTAL COMMON BILE DUCT/SDA\n  Contrast: OPTIRAY Amt: 20\n 
______________________________________________________________________________\n               

## Load the Parquet Dataset

In [6]:
df_inpatient = pl.read_parquet((inppatient_path / "parquet"))
cons.print(df_inpatient.schema)
cons.print(df_inpatient.shape)
cons.print(df_inpatient.head())

Schema({'hadm_id': Int64, 'comment': String, 'note_id': Int64, 'category': String, 'description': String, 'text': 
String, 'begin': Int64, 'end': Int64, 'covered_text': String, 'code': String, 'code_system': String, 
'code_description': String, 'type': String})

(7598, 13)

shape: (5, 13)
┌─────────┬─────────┬─────────┬───────────┬───┬─────────┬─────────────┬────────────────────┬───────┐
│ hadm_id ┆ comment ┆ note_id ┆ category  ┆ … ┆ code    ┆ code_system ┆ code_description   ┆ type  │
│ ---     ┆ ---     ┆ ---     ┆ ---       ┆   ┆ ---     ┆ ---         ┆ ---                ┆ ---   │
│ i64     ┆ str     ┆ i64     ┆ str       ┆   ┆ str     ┆ str         ┆ str                ┆ str   │
╞═════════╪═════════╪═════════╪═══════════╪═══╪═════════╪═════════════╪════════════════════╪═══════╡
│ 130520  ┆         ┆ 1250764 ┆ Radiology ┆ … ┆ BF10YZZ ┆ ICD-10-PCS  ┆ Fluoroscopy of     ┆ Human │
│         ┆         ┆         ┆           ┆   ┆         ┆             ┆ Bile Ducts usin…   ┆       │
│ 130520  ┆         ┆ 40383   ┆ Discharge ┆ … ┆ K83.1   ┆ ICD-10-CM   ┆ Obstruction of     ┆ Human │
│         ┆         ┆         ┆ summary   ┆   ┆         ┆             ┆ bile duct          ┆       │
│ 130520  ┆         ┆ 40383   ┆ Discharge ┆ … ┆ K83.1   ┆ ICD-10-CM   ┆ Obstruction of     ┆ Human │
│         ┆         ┆         ┆ summary   ┆   ┆         ┆             ┆ bile duct          ┆       │
│ 130520  ┆         ┆ 40383   ┆ Discharge ┆ … ┆ K86.1   ┆ ICD-10-CM   ┆ Other chronic      ┆ Human │
│         ┆         ┆         ┆ summary   ┆   ┆         ┆             ┆ pancreatitis       ┆       │
│ 130520  ┆         ┆ 40383   ┆ Discharge ┆ … ┆ 0FT40ZZ ┆ ICD-10-PCS  ┆ Resection of       ┆ Human │
│         ┆         ┆         ┆ summary   ┆   ┆         ┆             ┆ Gallbladder, Open… ┆       │
└─────────┴─────────┴─────────┴───────────┴───┴─────────┴─────────────┴────────────────────┴───────┘

In [8]:
df_profee = pl.read_parquet((profee_path / "parquet"))
cons.print(df_profee.schema)
cons.print(df_profee.shape)
cons.print(df_profee.head())

Schema({'hadm_id': Int64, 'comment': String, 'note_id': Int64, 'category': String, 'description': String, 'text': 
String, 'begin': Int64, 'end': Int64, 'covered_text': String, 'code': String, 'code_system': String, 
'code_description': String, 'type': String})

(5563, 13)

shape: (5, 13)
┌─────────┬─────────┬─────────┬───────────┬───┬────────┬─────────────┬─────────────────────┬───────┐
│ hadm_id ┆ comment ┆ note_id ┆ category  ┆ … ┆ code   ┆ code_system ┆ code_description    ┆ type  │
│ ---     ┆ ---     ┆ ---     ┆ ---       ┆   ┆ ---    ┆ ---         ┆ ---                 ┆ ---   │
│ i64     ┆ str     ┆ i64     ┆ str       ┆   ┆ str    ┆ str         ┆ str                 ┆ str   │
╞═════════╪═════════╪═════════╪═══════════╪═══╪════════╪═════════════╪═════════════════════╪═══════╡
│ 196522  ┆         ┆ 231448  ┆ ECG       ┆ … ┆ 93000  ┆ CPT         ┆ Electrocardiogram,  ┆ Human │
│         ┆         ┆         ┆           ┆   ┆        ┆             ┆ routine ECG…        ┆       │
│ 196522  ┆         ┆ 231448  ┆ ECG       ┆ … ┆ R94.31 ┆ ICD-10-CM   ┆ Abnormal            ┆ Human │
│         ┆         ┆         ┆           ┆   ┆        ┆             ┆ electrocardiogram   ┆       │
│         ┆         ┆         ┆           ┆   ┆        ┆             ┆ [EC…                ┆       │
│ 196522  ┆         ┆ 231447  ┆ ECG       ┆ … ┆ 93000  ┆ CPT         ┆ Electrocardiogram,  ┆ Human │
│         ┆         ┆         ┆           ┆   ┆        ┆             ┆ routine ECG…        ┆       │
│ 196522  ┆         ┆ 231447  ┆ ECG       ┆ … ┆ R94.31 ┆ ICD-10-CM   ┆ Abnormal            ┆ Human │
│         ┆         ┆         ┆           ┆   ┆        ┆             ┆ electrocardiogram   ┆       │
│         ┆         ┆         ┆           ┆   ┆        ┆             ┆ [EC…                ┆       │
│ 196522  ┆         ┆ 38962   ┆ Discharge ┆ … ┆ R07.9  ┆ ICD-10-CM   ┆ Chest pain,         ┆ Human │
│         ┆         ┆         ┆ summary   ┆   ┆        ┆             ┆ unspecified         ┆       │
└─────────┴─────────┴─────────┴───────────┴───┴────────┴─────────────┴─────────────────────┴───────┘

## Load [Code + Evidence] Splits

In [9]:
ip_test_split = Path("../data/mdace/Inpatient/MDace-code-ev-test.csv").read_text().splitlines()
ip_train_split = Path("../data/mdace/Inpatient/MDace-code-ev-train.csv").read_text().splitlines()
ip_valid_split = Path("../data/mdace/Inpatient/MDace-code-ev-val.csv").read_text().splitlines()

In [33]:
pr_test_split = Path("../data/mdace/Profee/MDace-code-ev-test.csv").read_text().splitlines()
pr_train_split = Path("../data/mdace/Profee/MDace-code-ev-train.csv").read_text().splitlines()
pr_valid_split = Path("../data/mdace/Profee/MDace-code-ev-val.csv").read_text().splitlines()

In [35]:
cons.print(f"Inpatient splits: train={len(ip_train_split)}, valid={len(ip_valid_split)}, test={len(ip_test_split)}")
cons.print(f"Profee splits: train={len(pr_train_split)}, valid={len(pr_valid_split)}, test={len(pr_test_split)}")

Inpatient splits: train=47900, valid=1691, test=3131

Profee splits: train=47750, valid=1641, test=3331